In [1]:
regions_dict = {
  1: 'Ленинградская область',
  3138: 'Санкт-Петербург',
  3268: 'Москва',
  3427: 'Волгоградская область',
  3902: 'Тульская область',
  4013: 'Омская область',
  4437: 'Краснодарский край',
  4882: 'Тюменская область',
  5188: 'Московская область',
}

In [5]:
import geopandas as gpd
from townsnet import Fetcher

for region_id, region_name in filter(lambda tpl : tpl[0] != 1, regions_dict.items()):
  path = f"data/regions/{region_id}/"
  districts = gpd.read_file(path + 'districts.geojson')
  crs = districts.estimate_utm_crs()
  print(region_name, crs)
  towns = Fetcher(units=districts).fetch_towns()
  towns.to_file(path + 'towns.geojson')
  print(len(towns))

Санкт-Петербург EPSG:32636
34
Москва EPSG:32637
232
Волгоградская область EPSG:32638


/home/vasilstar/townsnet/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 55 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


1296
Тульская область EPSG:32637


/home/vasilstar/townsnet/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 12 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


250
Омская область EPSG:32643


...

2766
Тюменская область EPSG:32642


/home/vasilstar/townsnet/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 83 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


1237
Московская область EPSG:32637


/home/vasilstar/townsnet/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 24 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


3077


In [3]:
from tqdm import tqdm
from townsnet import SERVICE_TYPES, ServiceType
from multiprocessing import Pool

for region_id, region_name in tqdm(regions_dict.items()):
  path = f"data/regions/{region_id}/"
  districts = gpd.read_file(path + 'districts.geojson')
  crs = districts.estimate_utm_crs()
  fetcher = Fetcher(units=districts)

  def foobar(st):
      try:
        services = fetcher.fetch_services(st)
        services.to_file(f'{path}services/{st.name}.geojson')
      except:
         ...
  
  with Pool(len(SERVICE_TYPES)) as pool:
    pool.map(foobar, [ServiceType(**service_type) for service_type in SERVICE_TYPES])

100%|██████████| 9/9 [01:32<00:00, 10.26s/it]


In [5]:
# import osmnx as ox

# for region_id, region_name in tqdm(regions_dict.items()):
#   path = f"data/regions/{region_id}/"
#   districts = gpd.read_file(path + 'districts.geojson')
#   geometry = districts.unary_union
#   crs = districts.estimate_utm_crs()
#   fetcher = Fetcher(units=districts)
#   graph = fetcher.fetch_graph()
#   ox.save_graphml(graph, path+'graph.graphml')

In [11]:
import osmnx as ox
from shapely import to_wkt
from dongraphio import DonGraphio, GraphType

for region_id, region_name in tqdm(filter(lambda tpl : tpl[0] != 1, regions_dict.items())):
  path = f"data/regions/{region_id}/"
  towns = gpd.read_file(path + 'towns.geojson')
  graph = ox.load_graphml(path + 'graph.graphml')

  crs = towns.estimate_utm_crs()
  towns = towns.to_crs(crs)
  graph = ox.project_graph(graph, crs)

  speed = 60 * 1000 / 60
  nodes, edges = ox.graph_to_gdfs(graph)
  edges['type'] = 'car'
  edges['time_min'] = edges['length'] / speed
  graph = ox.graph_from_gdfs(nodes,edges)

  for _,_,data in graph.edges(data=True):
    # data['type'] = 'car'
    data['geometry'] = to_wkt(data['geometry'])
    # data['time_min'] = data['length'] / speed

  dg = DonGraphio(crs.to_epsg(), graph)
  adj_mx = dg.get_adjacency_matrix(towns, towns, weight='time_min', graph_type=[GraphType.DRIVE])
  adj_mx.to_pickle(path + 'adj_mx.pickle')
  

0it [00:00, ?it/s]2024-06-12 17:14:30.668 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:84 - Creating adjacency matrix based on provided graph...
2024-06-12 17:14:33.942 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances ...
2024-06-12 17:14:34.092 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:93 - Adjacency matrix done!
1it [00:12, 12.70s/it]2024-06-12 17:14:45.238 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:84 - Creating adjacency matrix based on provided graph...
2024-06-12 17:14:47.917 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances ...
2024-06-12 17:14:49.770 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:93 - Adjacency matrix done!
2it [00:28, 14.45s/it]2024-06-12 17:15:23.574 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:84 - Creating adjacency matrix based on provided graph...
2024-06-12 17:15:31.682 | DEBUG    | dongraphio.base_model